In [ ]:
# Instalar paquetes
!pip install hubspot-api-client
!pip install rut_chile

In [ ]:
import requests
from rut_chile import rut_chile as rc
import pandas as pd
import numpy as np
import unicodedata
import re
import time
from datetime import datetime, timedelta

import time
import psutil
import os

# Abrir archivo de log
log_file = open("log.txt", "w")
log_file.write("--- INICIO DEL LOG ---\n")

start = time.time()
process = psutil.Process(os.getpid())
mem_start = process.memory_info().rss
# Guardar tiempo de inicio
inicio = time.time()


# Guardar tiempo de inicio
inicio = time.time()

# Función para validar rut
def validar_rut(rut):
    try:
        return rc.is_valid_rut(rut)
    except Exception as e:
        return False

def agregar_guion(valor):
    if pd.notna(valor) and len(valor) > 1:
        return valor[:-1] + '-' + valor[-1]
    else:
        return valor

# Función para eliminar 0 al principio de una cadena
def eliminar_cero(valor):
    if isinstance(valor, str) and valor.startswith('0'):
        return valor[1:]
    return valor

def consulta_api(id, access_token=None, identificador='rut'):
    # Definir URL y variables según identificador
    if identificador == 'rut':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/MedicoContratos/byRut/{id}/true'
    elif identificador == 'icm':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/MedicoContratos/byICM/{id}/true'
    elif identificador == 'direccion':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/Direcciones/byICM/{id}'
    elif identificador == 'datos_direccion':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/Direcc/byICM/{id}'

    # Obtener token si no se proporciona
    if access_token is None:
        access_token = autorizacion_api()

    # Configurar encabezados
    headers = {'Authorization': f'Bearer {access_token}'}

    try:
        # Realizar la solicitud
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Lanza una excepción para códigos HTTP >= 400

    except requests.exceptions.HTTPError as http_err:
        if response.status_code == 404:
            return {
                "error": "Recurso no encontrado.",
                "status": 404,
                "detalle": f"No se encontró el recurso solicitado con ID: {id}. URL: {url}"
            }
        elif response.status_code == 401:
            return {
                "error": "No autorizado.",
                "status": 401,
                "detalle": "El token es inválido o ha expirado. Intenta obtener un nuevo token."
            }
        else:
            return {
                "error": f"HTTP Error {response.status_code}",
                "detalle": response.text
            }
    except requests.exceptions.RequestException as req_err:
        return {
            "error": "Error de conexión.",
            "detalle": str(req_err)
        }

    # Si no hay errores, retorna el contenido como JSON
    return response.json()

def autorizacion_api():
    # URL de autenticación
    url_auth = 'https://authentication.colegiomedico.cl/api/authentication/authenticate'

    # Credenciales desde variables de entorno
    user = 'UsersCRSantiago'  # Valor por defecto
    psw = 'UsersCRSantiago2024_&_&'  # Valor por defecto

    # Verificar credenciales
    if not user or not psw:
        raise ValueError("Las credenciales de usuario o contraseña no están configuradas.")

    # Datos de autenticación
    data = {"userName": user, "password": psw}

    # Encabezados
    headers = {"Content-Type": "application/json"}

    # Solicitar token
    response_auth = requests.post(url_auth, headers=headers, json=data)

    # Verificar respuesta
    if response_auth.status_code != 200:
        raise Exception(f"Error al autenticar: {response_auth.status_code} - {response_auth.text}")

    # Retornar token
    access_token = response_auth.text
    return access_token.strip('"')

#recorrer la api del nacional desde un id_inical hasta un id_final
def obtener_datos_desde_api(id_inicial, id_final, identificador='icm'):
    resultados = []

    for id_actual in range(id_inicial, id_final + 1):
        try:
            log_file.write(f"Consultando ID: {id_actual}\n")
            # Llama a la función consulta_api (definida previamente)
            respuesta = consulta_api(str(id_actual), identificador=identificador)
            resultados.append(respuesta)

        except Exception as e:
            log_file.write(f"Error con ID {id_actual}: {e}\n")
            continue

    # Convertir resultados en un DataFrame
    return pd.DataFrame(resultados)


def limpiar_valor(valor):
    """Elimina caracteres especiales, convierte a minúsculas, y elimina tildes"""
    # Eliminar tildes y acentos
    valor_sin_tildes = unicodedata.normalize('NFKD', valor).encode('ASCII', 'ignore').decode('ASCII')
    # Eliminar caracteres especiales
    valor_limpio = re.sub(r'[^\w\s]', '', valor_sin_tildes)
    return valor_limpio.replace(' ', '_').lower()

# Función para validar email
def es_correo_valido(email):
    # Verificar si el email es NaN o vacío
    if pd.isna(email) or email.strip() == '':
        return False

    # Patrón de la expresión regular para un email válido
    patron = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

    # Usar re.match para comprobar si el email coincide con el patrón
    if re.match(patron, email):
        return True
    else:
        return False

def obtener_direcciones(df_o_lista):
    resultados = []
    contador = 0  # Contador de consultas

    # Verificar si la entrada es un DataFrame o una lista
    if isinstance(df_o_lista, pd.DataFrame):
        if 'idMedico' not in df_o_lista.columns:
            log_file.write("Error: El DataFrame no contiene la columna 'idMedico'.\n")
            return pd.DataFrame()
        ids = df_o_lista['idMedico'].dropna().unique()
    elif isinstance(df_o_lista, list):
        ids = list(set(df_o_lista))  # Eliminar duplicados
    else:
        log_file.write("Error: La entrada debe ser un DataFrame o una lista de IDs.\n")
        return pd.DataFrame()

    total_ids = len(ids)
    log_file.write(f"Total de IDs a procesar: {total_ids}\n")

    for id_medico in ids:
        contador += 1  # Incrementa el contador por cada consulta

        try:
            datos = consulta_api(id_medico, identificador='direccion')
        except Exception as e:
            log_file.write(f"Error en la consulta de ID {id_medico}: {e}\n")
            continue

        # Depuración: Mostrar progreso y datos recibidos
        log_file.write(f"Consulta {contador}/{total_ids} - ID: {id_medico}, Tipo de datos retornado: {type(datos)}\n")

        if isinstance(datos, list):
            resultados.extend(datos)
        else:
            log_file.write(f"Advertencia: La API devolvió un {type(datos)} en lugar de una lista para ID {id_medico}\n")

    # Convertir los resultados a un DataFrame si hay datos válidos
    df_resultado_direcciones = pd.DataFrame(resultados) if resultados else pd.DataFrame()

    log_file.write(f"Consultas finalizadas. Total procesados: {contador}, Filas obtenidas: {df_resultado_direcciones.shape[0]}\n")

    return df_resultado_direcciones

# Función para determinar el valor de 'email_final' basado en las reglas dadas
def determinar_email_final(row):
    if row['email_nulo_o_vacio']:
        return 'Nulo'
    elif not row['email_valido']:
        return 'Invalido'
    elif row['email_duplicado']:
        return 'Duplicado'
    else:
        return 'Valido'

#obtengo información de la API del nacional a través del rut, entregandole un df y la columna con id
def obtener_datos_desde_api_rut(df_ids, columna_id, identificador='rut'):
    resultados = []
    total_ids = len(df_ids)

    for idx, id_actual in enumerate(df_ids[columna_id], start=1):
        try:
            # Llama a la función consulta_api (definida previamente)
            respuesta = consulta_api(str(id_actual), identificador=identificador)
            resultados.append(respuesta)

            # Mostrar el progreso
            log_file.write(f"Procesando ID: {id_actual} ({idx}/{total_ids})\n")

        except Exception as e:
            log_file.write(f"Error con ID {id_actual}: {e}\n")
            continue

    # Añadir una nueva línea al finalizar para evitar que el último mensaje de progreso
    # sobrescriba el siguiente output en la consola
    log_file.write("\n")

    # Convertir resultados en un DataFrame
    return pd.DataFrame(resultados)

In [ ]:
from hubspot import HubSpot
from hubspot.crm.contacts.exceptions import ApiException
import os

api_client = HubSpot()
api_client.access_token = os.getenv('HUBSPOT_API_KEY')

#Especifica las propiedades que deseas recuperar
properties = [ 'address','firstname','segundo_nombre', 'lastname','apellido_materno', 'condicion', 'createdate','crs',
            'date_of_birth','direccion_de_trabajo','edad','email','estado_pago','fecha_inscripcion_al_colegio',
'fecha_titulo','gender','graduation_date','mobilephone','rcm','rut', 'lastmodifieddate', 'lifecyclestage',
              'phone','hs_object_source_label','especialidades','fecha_act_super','fecha_actualizacion_pagos','condicion_vital']

try:
    # Recupera todos los contactos con las propiedades especificadas
    all_contacts = api_client.crm.contacts.get_all(properties=properties)

    # Convierte cada contacto a un diccionario y almacenarlos en una lista
    contacts_list = [contact.to_dict() for contact in all_contacts]

    # Crear un DataFrame de pandas a partir de la lista de diccionarios
    df_contacts = pd.DataFrame(contacts_list)

except ApiException as e:
    log_file.write(f"Exception when calling contacts API: {e}\n")

# Normalizar la columna que contiene diccionarios
df_col_dict = pd.json_normalize(df_contacts['properties'])

# Opcional: unir el DataFrame normalizado con el DataFrame original, excluyendo la columna original
df_final_contactos = df_contacts.drop(columns=['properties']).join(df_col_dict)

# Ordenar por 'Fecha de creación' (descendente) y 'Rut_Normalizado' (ascendente)
df_final_contactos = df_final_contactos.sort_values(by=['createdate'], ascending = True)

In [ ]:
#normalizar columna rut
df_final_contactos['rut_normalizado'] = (
    df_final_contactos['rut'].str.replace(r'[\s.,-]', '', regex=True).apply(agregar_guion).apply(eliminar_cero).astype(str))

# Aplicar la función is_valid_rut y crear una nueva columna con los resultados
df_final_contactos['rut_valido'] = df_final_contactos['rut_normalizado'].apply(validar_rut)

# Extraer el RUT sin el dígito verificador y agregarlo a una nueva columna usando
df_final_contactos['rut_sdv'] = df_final_contactos['rut_normalizado'].str.split('-').str[0]

# Filtrar las filas donde 'rut' es True
df_rut_filtrado_crm = df_final_contactos[df_final_contactos['rut_valido'] == True]

columnas_hubspot =['id','rut_normalizado','rut','rcm','rut_sdv','fecha_actualizacion_pagos','email','mobilephone','condicion_vital','created_at','estado_pago','condicion']
df_hubspot = df_rut_filtrado_crm[columnas_hubspot]
df_rut_filtrado_crm = df_rut_filtrado_crm[columnas_hubspot]

In [ ]:
# Asegurarse de que la columna 'fecha_actualizacion_pagos' esté en formato de fecha
df_rut_filtrado_crm['fecha_actualizacion_pagos'] = pd.to_datetime(df_rut_filtrado_crm['fecha_actualizacion_pagos'], format='%Y-%m-%d', errors='coerce')

# Definir la fecha límite (30 días atrás)
fecha_limite = datetime.now() - timedelta(days=30)

#filtrar usuario vivos o sin condicion vital
df_rut_filtrado_crm = df_rut_filtrado_crm[((df_rut_filtrado_crm['estado_pago'].isna()) | (df_rut_filtrado_crm['estado_pago'].fillna('') == 'Moroso')) & (df_rut_filtrado_crm['condicion_vital'] != 'FALLECIDO') & (df_rut_filtrado_crm['condicion'] == 'Afiliado') ]





In [ ]:
df_ruts = df_rut_filtrado_crm

In [ ]:
import concurrent.futures
import pandas as pd
import sys

def obtener_datos_desde_api_rut(df, columna_id, identificador='rut'):
    resultados = []
    total_ids = len(df)

    def consultar_id(id_actual):
        try:
            resultado = consulta_api(str(id_actual), identificador=identificador)
            log_file.write(f"Procesado: {id_actual}\n")
            return resultado
        except Exception as e:
            log_file.write(f"Error con {id_actual}: {e}\n")
            return None

    with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
        for i, resultado in enumerate(executor.map(consultar_id, df[columna_id])):
            if resultado is not None:
                resultados.append(resultado)

            if i % 10 == 0 or i == total_ids - 1:  # Mostrar cada 10 registros
                log_file.write(f"Progreso: {i+1}/{total_ids} IDs procesados\n")

    return pd.DataFrame(resultados)




In [ ]:
# Llamar a la función utilizando la columna 'id' como identificadores
resultados_rut = obtener_datos_desde_api_rut(df_ruts, columna_id='rut_sdv')

In [ ]:
resultados_rut = resultados_rut[resultados_rut['rut'].notna()]

In [ ]:
# Manejar valores nulos en la columna 'rut' (opcional)
resultados_rut['rut'] = resultados_rut['rut'].fillna(0)  # Rellena nulos con 0
resultados_rut['idMedico'] = resultados_rut['idMedico'].fillna(0)
# Convertir la columna 'rut' a enteros
resultados_rut['rut'] = resultados_rut['rut'].astype(int)
resultados_rut['idMedico'] = resultados_rut['idMedico'].astype(int)
# Guardar tiempo de inicio
OBTENER_INFO = time.time()

In [ ]:
resultados_rut

In [ ]:
resultados_rut.rename(columns={'rut': 'rut_sdv'}, inplace=True)
df_hubspot.rename(columns={'rut_sdv': 'rut_sdv'}, inplace=True)

resultados_rut['rut_sdv'] = resultados_rut['rut_sdv'].astype(str)
df_hubspot['rut_sdv'] = df_hubspot['rut_sdv'].astype(str)

df_cruce = pd.merge(resultados_rut, df_hubspot, on='rut_sdv', how='inner')

# reemplazar nan con vacio
df_cargainformacion = df_cruce.fillna('')

In [ ]:
df_cargainformacion

OPCIONES REGIONAL Y PAIS DE EJERCICIO

In [ ]:
# Filtrar valores que no sean NaN y que sean de tipo string
opciones_regional = sorted([x for x in resultados_rut['descConsejoRegional'].unique() if isinstance(x, str) and x is not np.nan])

#Generar opciones para regionales
opciones_regional = [{"label": item, "value": limpiar_valor(item)} for item in opciones_regional]


opciones_pais = sorted([x for x in resultados_rut['descPaisDeEjercicio'].unique() if isinstance(x, str) and x is not np.nan])
#Generar opciones para pais

opciones_pais = [{"label": item, "value": limpiar_valor(item)} for item in opciones_pais]

opciones_pais

In [ ]:
##actualizar y crear opciones regionales

from hubspot import HubSpot
from hubspot.crm.properties import ApiException, PropertyUpdate, Option
import os

# Token de acceso
access_token = os.getenv('HUBSPOT_API_KEY')

# Inicializar el cliente de HubSpot
api_client = HubSpot(access_token=access_token)

# Nombre de la propiedad
property_name = 'afiliacion_regional'

# Nuevas opciones a agregar (asegúrate de que la variable opciones esté definida)
nuevas_opciones = opciones_regional

try:
    # Obtener la propiedad actual
    properties_api = api_client.crm.properties
    property_data = properties_api.core_api.get_by_name(object_type='contacts', property_name=property_name)

    # Obtener opciones actuales
    current_options = property_data.options if property_data.options else []

    # Crear un conjunto de valores existentes para evitar duplicados
    valores_existentes = {option.value for option in current_options}

    # Añadir nuevas opciones a las opciones existentes
    for option in nuevas_opciones:
        if option['value'] not in valores_existentes:
            new_option = Option(label=option['label'], value=option['value'])
            current_options.append(new_option)
            valores_existentes.add(option['value'])  # Actualizar el conjunto de valores existentes
            log_file.write(f"{option}\n")
    # Preparar datos para la actualización
    update_data = PropertyUpdate(
        label=property_data.label,
        options=current_options,
        group_name=property_data.group_name,
        type=property_data.type,
        field_type=property_data.field_type    )

    # Actualizar la propiedad
    properties_api.core_api.update(object_type='contacts', property_name=property_name, property_update=update_data)

    log_file.write("Propiedad actualizada con éxito\n")

except ApiException as e:
    log_file.write(f"Error al conectar con la API de HubSpot: {e}\n")
except Exception as e:
    log_file.write(f"Error inesperado: {e}\n")

In [ ]:
##actualizar y crear opciones pais

from hubspot import HubSpot
from hubspot.crm.properties import ApiException, PropertyUpdate, Option
import os

# Token de acceso
access_token = os.getenv('HUBSPOT_API_KEY')

# Inicializar el cliente de HubSpot
api_client = HubSpot(access_token=access_token)

# Nombre de la propiedad
property_name = 'pais_ejercicio'

# Nuevas opciones a agregar (asegúrate de que la variable opciones esté definida)
nuevas_opciones = opciones_pais

try:
    # Obtener la propiedad actual
    properties_api = api_client.crm.properties
    property_data = properties_api.core_api.get_by_name(object_type='contacts', property_name=property_name)

    # Obtener opciones actuales
    current_options = property_data.options if property_data.options else []

    # Crear un conjunto de valores existentes para evitar duplicados
    valores_existentes = {option.value for option in current_options}

    # Añadir nuevas opciones a las opciones existentes
    for option in nuevas_opciones:
        if option['value'] not in valores_existentes:
            new_option = Option(label=option['label'], value=option['value'])
            current_options.append(new_option)
            valores_existentes.add(option['value'])  # Actualizar el conjunto de valores existentes
            log_file.write(f"{option}\n")
    # Preparar datos para la actualización
    update_data = PropertyUpdate(
        label=property_data.label,
        options=current_options,
        group_name=property_data.group_name,
        type=property_data.type,
        field_type=property_data.field_type    )

    # Actualizar la propiedad
    properties_api.core_api.update(object_type='contacts', property_name=property_name, property_update=update_data)

    log_file.write("Propiedad actualizada con éxito\n")

except ApiException as e:
    log_file.write(f"Error al conectar con la API de HubSpot: {e}\n")
except Exception as e:
    log_file.write(f"Error inesperado: {e}\n")

In [ ]:
# Crear un mapeo para normalizar valores
mapeo_estado_civil = {
    None: '',  # Opcional: Define qué hacer con None
    float('nan'): '',  # Manejo explícito de NaN
    '...': '',  # Corregir valores inválidos
    'SOLTERA(O)': 'SOLTERA(O)',  # Valores válidos
    'CASADA(O)': 'CASADA(O)',
    'VIUDA(O)': 'VIUDA(O)'}# Valores válidos}

# Aplicar mapeo a la columna estado_civil
df_cargainformacion['descEstadoCivil'] = df_cargainformacion['descEstadoCivil'].map(mapeo_estado_civil)
# reemplazar nan con vacio
df_cargainformacion = df_cargainformacion.fillna('')

# Aplicar la función limpiar_valor solo a los valores que no son NaN
df_cargainformacion['descPaisDeEjercicio'] = df_cargainformacion['descPaisDeEjercicio'].apply(lambda x: limpiar_valor(x) if pd.notna(x) else x)
df_cargainformacion['descConsejoRegional'] = df_cargainformacion['descConsejoRegional'].apply(lambda x: limpiar_valor(x) if pd.notna(x) else x)



In [ ]:
#Actualizar información

import requests
import os

# Configura tu API Key de HubSpot
api_key = os.getenv('HUBSPOT_API_KEY')

# Supongamos que tienes un DataFrame 'df_contactos' con las columnas: 'id', 'firstname', 'segundo_nombre', 'lastname', 'apellido_materno'
df_contactos = df_cargainformacion


# Define los headers
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

# Itera sobre cada fila del DataFrame para actualizar el contacto en HubSpot
contador = 0
for index, row in df_contactos.iterrows():
    contact_id = row['id']
    data = {
        "properties": {
            "condicion_vital": row['descCondicionVital'],
            "gender": row['descSexo'],
            "afiliacion_regional": row['descConsejoRegional'],
            "marital_status": row['descEstadoCivil'],
            "pais_ejercicio": row['descPaisDeEjercicio']
        }
    }

    # Define la URL de la API de HubSpot para actualizar un contacto
    url = f'https://api.hubapi.com/crm/v3/objects/contacts/{contact_id}'

    # Realiza la solicitud PATCH para actualizar el contacto
    response = requests.patch(url, headers=headers, json=data)

    # Verifica la respuesta
    if response.status_code == 200:
        log_file.write(f"Contacto con ID {contact_id} actualizado exitosamente. \n")
    else:
        log_file.write(f'Error al actualizar el contacto con ID {contact_id}: {response.status_code}\n')
        log_file.write(f"{response.text}\n")
    contador += 1
    # Mostrar el progreso en la misma línea
    log_file.write(f"Progreso: {contador}/{len(df_contactos)}\n")
contador = 0 # arreglar el largo del string

In [ ]:
columnas_x = [col for col in df_cargainformacion.columns if col.endswith('_x')]
columnas_y = [col for col in df_cargainformacion.columns if col[:-2] + '_y' in df_cargainformacion.columns]

for col_x in columnas_x:
    col_base = col_x[:-2]  # Nombre sin _x
    df_cargainformacion[col_base] = df_cargainformacion[col_x]  # Copia valores

df_cargainformacion.drop(columns=columnas_x + columnas_y, inplace=True, errors='ignore')

In [ ]:
df_cargainformacion

Informacion Dinamica

In [ ]:

df_cargainformacion = df_cargainformacion.dropna(subset=['contratos'])  # Eliminar NaN antes de expandir
df_cargainformacion = df_cargainformacion[df_cargainformacion['contratos'].apply(lambda x: isinstance(x, list))]

df_expandido = df_cargainformacion.explode('contratos')
df_expandido = pd.concat([df_expandido.drop(columns=['contratos']), df_expandido['contratos'].apply(pd.Series)], axis=1)

In [ ]:
df_expandido

In [ ]:
df_expandido = df_expandido.loc[:, ~df_expandido.columns.duplicated()]

In [ ]:
df_expandido['descEstadoPago']

In [ ]:
df_expandido.rename(columns={'idMedico': 'idMedico'}, inplace=True)
df_hubspot.rename(columns={'rcm': 'idMedico'}, inplace=True)

df_expandido['idMedico'] = df_expandido['idMedico'].astype(str)
df_hubspot['idMedico'] = df_hubspot['idMedico'].astype(str)

df_cruce_pagos = pd.merge(df_expandido, df_hubspot, on='idMedico', how='inner')

columnas_df = ['idMedico','descEstamento', 'descCondicionSocio', 'fechaInscripcion',
               'descEstadoPago','id']
df_expandido = df_expandido[columnas_df]
#df_expandido['descEstadoPago'].unique()


# Crear un mapeo para normalizar valores
mapeo_estado_pago = {
    'AL DIA': 'Al día',  # Corregir valores inválidos
    'LIBERADO': 'Liberado',  # Valores válidos
    'LIBERADO DIRECTORIO FALMED': 'Liberado',
    'FALMED SENIOR LIBERADO': 'Liberado',
    'MOROSO12': 'Moroso',
    'MOROSO' : 'Moroso',
    'None' : ''}# Valores válidos

# Aplicar mapeo a la columna estado_civil
df_expandido['descEstadoPago'] = df_expandido['descEstadoPago'].map(mapeo_estado_pago)

# Crear un mapeo para normalizar valores
mapeo_condicion_socio = {
    'AFILIADO': 'Afiliado',  # Corregir valores inválidos
    'DESAFILIADO': 'Desafiliado',  # Valores válidos
    'RENUNCIADO': 'Renunciado',
    'REINSCRITO': 'Afiliado',
    'None' : ''}# Valores válidos

# Aplicar mapeo a la columna estado_civil
df_expandido['descCondicionSocio'] = df_expandido['descCondicionSocio'].map(mapeo_condicion_socio)
# Convertir la columna 'rut' a enteros
#df_expandido['idMedico'] = df_expandido['idMedico'].astype(int)
df_expandido['idMedico'].unique()
# Conversión directa a int con manejo de errores
df_expandido['idMedico'] = pd.to_numeric(df_expandido['idMedico'], errors='coerce').astype('Int64')
df_expandido_final = df_expandido.drop_duplicates(subset=["idMedico", "descEstamento"], keep="last")


In [ ]:
df_expandido_final

In [ ]:
# Convertir la columna 'fecha_antecedente' al formato datetime
df_expandido_final['fechaInscripcion'] = pd.to_datetime(df_expandido_final['fechaInscripcion'], format='%d-%m-%Y')
df_expandido_final

In [ ]:
# Transformar el DataFrame: filas -> columnas
df_expandido_final = df_expandido_final.reset_index()
df_expandido_final = df_expandido_final.pivot(index='idMedico', columns='descEstamento', values=["descCondicionSocio", "fechaInscripcion", "descEstadoPago","id"])
# Ajustar nombres de las columnas
#df_expandido_final.columns = ['_'.join(col).strip() for col in df_expandido_final.columns.values]
df_expandido_final.columns = ['_'.join(map(str, col)).strip() for col in df_expandido_final.columns.values]

df_expandido_final.reset_index(inplace=True)


In [ ]:
df_expandido_final

In [ ]:
# crear nuevo valor para fecha_actualizacion pago


df_expandido_final['fecha_actualizacion_pagos'] = pd.to_datetime('today').normalize()


# Convertir la columna 'fecha_antecedente' al formato datetime
df_expandido_final['fechaInscripcion_COLEGIO'] = pd.to_datetime(df_expandido_final['fechaInscripcion_COLEGIO'], format='%d-%m-%Y')
df_expandido_final['fecha_actualizacion_pagos'] = pd.to_datetime(df_expandido_final['fecha_actualizacion_pagos'], format='%d-%m-%Y')

# Convertir el Timestamp a un string en el formato adecuado
df_expandido_final['fecha_actualizacion_pagos'] = df_expandido_final['fecha_actualizacion_pagos'].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None)
df_expandido_final['fechaInscripcion_COLEGIO'] = df_expandido_final['fechaInscripcion_COLEGIO'].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None)
# Reemplazar los NaN en la columna 'afiliacion_regional' por ""
if 'descCondicionSocio_CLUB CAMPO' not in df_expandido_final.columns:
    df_expandido_final['descCondicionSocio_CLUB CAMPO'] = np.nan  # O cualquier valor que desees para la columna vacía
if 'descEstadoPago_CLUB CAMPO' not in df_expandido_final.columns:
    df_expandido_final['descEstadoPago_CLUB CAMPO'] = np.nan
if 'descCondicionSocio_FATMED' not in df_expandido_final.columns:
    df_expandido_final['descCondicionSocio_FATMED'] = np.nan  # O cualquier valor que desees para la columna vacía
if 'descEstadoPago_FATMED' not in df_expandido_final.columns:
    df_expandido_final['descEstadoPago_FATMED'] = np.nan

df_expandido_final['descCondicionSocio_CLUB CAMPO'] = df_expandido_final['descCondicionSocio_CLUB CAMPO'].fillna("SIN_AFILIACION")
df_expandido_final['descEstadoPago_CLUB CAMPO'] = df_expandido_final['descEstadoPago_CLUB CAMPO'].fillna("SIN_AFILIACION")
# Reemplazar "Al Dia" por "Al día" en la columna 'estado_pago'
df_expandido_final['descEstadoPago_CLUB CAMPO'] = df_expandido_final['descEstadoPago_CLUB CAMPO'].str.replace('Al día', 'AL_DIA', regex=False)
df_expandido_final['descEstadoPago_CLUB CAMPO'] = df_expandido_final['descEstadoPago_CLUB CAMPO'].str.upper()
df_expandido_final['descCondicionSocio_CLUB CAMPO'] = df_expandido_final['descCondicionSocio_CLUB CAMPO'].str.upper()

# Ajustar opciones de pago de otras afiliaciones
df_expandido_final['descEstadoPago_FSG'] = df_expandido_final['descEstadoPago_FSG'].apply(lambda x: limpiar_valor(x) if pd.notna(x) else x)
df_expandido_final['descEstadoPago_FALMED'] = df_expandido_final['descEstadoPago_FALMED'].apply(lambda x: limpiar_valor(x) if pd.notna(x) else x)
df_expandido_final['descEstadoPago_FATMED'] = df_expandido_final['descEstadoPago_FATMED'].apply(lambda x: limpiar_valor(x) if pd.notna(x) else x)
df_expandido_final['descEstadoPago_FSG'] = df_expandido_final['descEstadoPago_FSG'].fillna("sin_afiliacion")
df_expandido_final['descEstadoPago_FALMED'] = df_expandido_final['descEstadoPago_FALMED'].fillna("sin_afiliacion")
df_expandido_final['descEstadoPago_FATMED'] = df_expandido_final['descEstadoPago_FATMED'].fillna("sin_afiliacion")

# reemplazar nan con vacio
df_expandido_final = df_expandido_final.fillna('')

columnas_df = ['id_COLEGIO','fechaInscripcion_COLEGIO','descCondicionSocio_COLEGIO','descEstadoPago_COLEGIO',
                   'descCondicionSocio_CLUB CAMPO','descEstadoPago_CLUB CAMPO','fecha_actualizacion_pagos','descEstadoPago_FSG', 'descEstadoPago_FALMED', 'descEstadoPago_FATMED']

df_expandido_final[columnas_df]

In [ ]:
import requests
import time
import sys
from tqdm import tqdm  # Barra de progreso
import os

# Configura tu API Key de HubSpot
api_key = os.getenv('HUBSPOT_API_KEY')

# Selecciona los primeros 200 contactos
df_contactos = df_expandido_final

# Define los headers
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

# Itera sobre cada fila del DataFrame con barra de progreso
# Mantén un margen de tiempo para evitar saturar la API
for index, row in tqdm(df_contactos.iterrows(), total=len(df_contactos), desc="Actualizando contactos", ncols=100):
    try:
        contact_id = row['id_COLEGIO']
        data = {
            "properties": {
                "fecha_inscripcion_al_colegio": row.get('fechaInscripcion_COLEGIO', ""),
                "condicion": row.get('descCondicionSocio_COLEGIO', ""),
                "estado_pago": row.get('descEstadoPago_COLEGIO', ""),
                "club_medico": row.get('descCondicionSocio_CLUB CAMPO', ""),
                "estado_pago_club_medico": row.get('descEstadoPago_CLUB CAMPO', ""),
                "fecha_actualizacion_pagos": row.get('fecha_actualizacion_pagos', ""),
                "estado_pago_fsg": row.get('descEstadoPago_FSG', ""),
                "estado_pago_falmed": row.get('descEstadoPago_FALMED', ""),
                "estado_pago_fatmed": row.get('descEstadoPago_FATMED', "")
            }
        }

        # URL de la API de HubSpot
        url = f'https://api.hubapi.com/crm/v3/objects/contacts/{contact_id}'

        # Realiza la solicitud PATCH
        response = requests.patch(url, headers=headers, json=data)

        # Verifica la respuesta
        if response.status_code == 200:
            # El contacto fue actualizado correctamente
            log_file.write(f"✅ Contacto {contact_id} actualizado correctamente.\n")
        elif response.status_code == 429:
            # Límite de tasa alcanzado, espera antes de reintentar
            wait_time = int(response.headers.get('X-RateLimit-Reset', 5))  # Establecer tiempo de espera
            log_file.write(f"🚨 Límite de tasa alcanzado. Esperando {wait_time} segundos...\n")
            time.sleep(wait_time)
        else:
            log_file.write(f"❌ Error al actualizar {contact_id}: {response.status_code} - {response.text}\n")

    except Exception as e:
        log_file.write(f"⚠ Error en el contacto {contact_id}: {str(e)}\n")

    # Pausa para evitar saturar la API
    time.sleep(0.3)  # Ajusta según tus necesidades


In [ ]:
mem_end = process.memory_info().rss
end = time.time()

log_file.write(f"Tiempo total: {end - start:.2f} segundos\n")
log_file.write(f"Memoria total usada: {(mem_end - mem_start)/1024**2:.2f} MB\n")

# Cerrar el archivo de log
log_file.write("--- FIN DEL LOG ---\n")
log_file.close()